In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
from IPython.display import clear_output

In [2]:
#!/usr/bin/env python
# coding: utf-8

"""CMOS MT9V011 Cluster Detection"""

#Import libraries
import os
import sys
import fnmatch
import pickle
import pandas as pd
from math import sqrt
from tqdm import tqdm
from skimage.feature import blob_log

import numpy as np
from numpy import interp

In [ ]:
#Files directory
#DATADIR = "/nfs/NASPG/BTData/Jul2012_CMOS_data/MT9V011_Firenze_2012_07_13/TEST/"
DATADIR = "/nfs/NASPG/BTData/Jul2012_CMOS_data/MT9V011_Firenze_2012_07_13/16_MT9V011_017_G01_050ms_3MeV_-60/"
#DATADIR = sys.argv[1]

#Initialize pedestal array and clusters dataframe
PEDS = np.zeros((307200,1))
PEDS[: , :] = np.nan #initialize to NaN so they can be easily excluded from mean calculation
CLUSTERS = pd.DataFrame({"Max":[], "Sum":[], "Size":[], "row":[],"column":[]})


In [ ]:
#Evaluate threshold for pedestal calculation
if not os.path.isfile(DATADIR + "pedestals.npy"):
    RAW_VALUES = np.array(0)
    for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR),
                                                '*.txt'))):
        clear_output(wait=True)
        print("Processing " + _filename + " for threshold computation")
        RAW_VALUES = np.append(RAW_VALUES, np.loadtxt(DATADIR + _filename, dtype="int"))
        #threshold is based on distribution of all values, assuming signal pixel are a minority on every frame
        THRESH = (RAW_VALUES.mean() + 2 * RAW_VALUES.std()).astype(int) 

In [ ]:
#Calculate or (load) Pedestals
if not os.path.isfile(DATADIR + "pedestals.npy"):
    for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR),
                                                '*.txt'))):
        clear_output(wait=True)
        print("Processing " + _filename + " for pedestals computation")
        im = np.loadtxt(DATADIR + _filename)
        if im.shape == (307200,):#check if frame is complete
            mask = im < THRESH  # mask of pixels under threshold (good for pedestals)
            im[im >= THRESH] = np.nan  # to prevent adding events with signal we set them to NaN
            im = np.atleast_2d(im).T #to force a 2D array
            PEDS = np.concatenate((PEDS, im), axis=1) #each pixel as a value from all the frames
        else:
            continue
    #calculate mean and std of each pixel for all the frames ignoring masked values
    PEDS = np.array([np.nanmean(PEDS,axis=1), np.nanstd(PEDS,axis=1)]) 
    PEDS.dump(DATADIR + "pedestals.npy")
else:
    PEDS = np.load(DATADIR + "pedestals.npy", allow_pickle="True")

In [ ]:
#Find clusters in every frame
for _filename in tqdm(sorted(fnmatch.filter(os.listdir(DATADIR), '*.txt'))):
    clear_output(wait=True)
    print("Processing " + _filename)
    frame = np.loadtxt(DATADIR + _filename, dtype="int")

    if frame.shape == PEDS[0].shape: #check if frame is complete
        frame = frame - PEDS[0] #pedestal subtraction
        frame[np.isnan(frame)] = 0 #convert NaN values to 0 (needed for skimage library)
    else:
        continue

    frame = frame.reshape(480, 640) #reshape array into VGA image
    _temp = interp(frame, [0, 1024], [0, 1]) #convert image to grayscale
    
    #Find "blobs" with Lagrangian of Gaussian algorithm of Skimage library
    blobs_log = blob_log(_temp, max_sigma=50, num_sigma=50, threshold=.05)
    blobs_log[:, 2] = blobs_log[:, 2] * 2 * sqrt(2)
    
    for _x, _y, _r in blobs_log:
        _min_y = (_y - _r).astype(int)
        _max_y = (_y + _r).astype(int)
        _min_x = (_x - _r).astype(int)
        _max_x = (_x + _r).astype(int)
        _subim = frame[_min_x:_max_x, _min_y:_max_y]#retrieve subarray of original value based on Skimage blob location
        if np.nanmax(_subim) > 2 * PEDS[1][np.nanargmax(_subim)]:#check if max of subimage is "seed" of a cluster (based on ped std of that pixel)
            CLUSTERS = CLUSTERS.append({'Max': np.nanmax(_subim), 'Sum' : np.nansum(_subim),
                                        'Size' : _subim.size,
                                        'row' : _x,
                                        'column' : _y},
                                       ignore_index=True)#add cluster info to DataFrame
    
    CLUSTERS.to_feather(DATADIR + "clusters.feather")#save clusters to file

In [ ]:
CLUSTERS